**DATA AUGMENTATION**

In [1]:
import numpy as np
import pandas as pd
import pickle
import os
from PIL import Image
import math
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.image import img_to_array, array_to_img, load_img

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# !pip install pydrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Using TensorFlow backend.


In [0]:
path = "gdrive/My Drive/NeuralNetCapstone/"
picklefile = "threechannels.pkl"

with open(os.path.join(path,picklefile), "rb") as f:
    images = pickle.load(f)

print ("Shape:",images.shape)

Shape: (51759, 128, 128, 3)


In [0]:
path = "gdrive/My Drive/NeuralNetCapstone/"
#Get y labels from datafile
datasheetfile = "final_df.csv"
df = pd.read_csv(os.path.join(path,datasheetfile))

In [0]:
all_classes = df.columns.values[-14:]

In [0]:
y_true = df[all_classes].values

In [0]:
label_counts = list()
for idx, each in enumerate(all_classes):
    idxs = df[df['Finding Labels'].str.contains(each)].index
    label_counts.append(len(idxs))

# fig, ax1 = plt.subplots(1,1,figsize = (12, 8))
# ax1.bar(np.arange(len(label_counts))+0.5, label_counts)
# ax1.set_xticks(np.arange(len(label_counts))+0.5)
# _ = ax1.set_xticklabels(label_counts.index, rotation = 90)

total = np.sum(label_counts)
ratio = [math.ceil((1 - (x/total))*3000) for x in label_counts]
print (label_counts)
print (ratio)

[11559, 2776, 4667, 2303, 13317, 2516, 1686, 227, 19894, 5782, 6331, 3385, 1431, 5302]
[2573, 2898, 2828, 2915, 2508, 2908, 2938, 2992, 2265, 2787, 2767, 2875, 2948, 2805]


In [0]:
# #Extract Hernia cases
# idxs = df[df['Finding Labels'].str.contains('Hernia')].index
# images_hernia = images[idxs.values]

#Extract Pneumonia cases
idxs = df[df['Finding Labels'].str.contains('Pneumonia')].index
images_pneumonia = images[idxs.values]


In [0]:
images_pneumonia.shape

(1431, 128, 128, 3)

In [0]:
# Free memory
import gc
del images
gc.collect()

0

In [0]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


In [0]:
for idx, each in enumerate(all_classes):
    idxs = df[df['Finding Labels'].str.contains(each)].index
    this_ratio = ratio[idx]
    image_idxs = np.random.choice(idxs,(this_ratio//4))
    images_class = images[image_idxs]
        
    for x in images_class:        
        x = x.reshape((1,) + x.shape)
        i = 0
        for batch in datagen.flow(x, batch_size=1,
                                  save_to_dir='preview', save_prefix=each, save_format='jpg'):
            i += 1
            if i > 3:
                break  
    print (each+": done")


In [0]:
added_images = list()
labels = list()
classes = list(all_classes)
for each in os.listdir('preview')[15000:]:
    this_class = None
    if each.split('_')[0] == 'Pleural':
        this_class = classes.index(each.split('_')[0] +'_'+ each.split('_')[1] )
    else:
        this_class = classes.index(each.split('_')[0])
    img = load_img('preview/'+each)  # this is a PIL image
    x = img_to_array(img)
    this_label = [0]*14
    this_label[this_class] = 1
    labels.append(this_label)
    added_images.append(x)
added_images = np.array(added_images)
labels = np.array(labels)

(15000, 128, 128, 3)

(15000, 14)

In [0]:
!mkdir preview
# !rm -r preview/

In [0]:
h5f = h5py.File(path + 'added_images_2.h5', 'w')
h5f.create_dataset('dataset_1', data=added_images)
h5f.close()

h5f = h5py.File(path + 'added_labels_2.h5', 'w')
h5f.create_dataset('dataset_1', data=labels)
h5f.close()

In [0]:
#Create Numpy array

stack = list()
for each in os.listdir('preview_pneumonia'):
    img = load_img(os.path.join('preview_pneumonia',each)) 
    x = img_to_array(img)
    stack.append(x)

stack = np.array(stack)
# with open(path + 'pneumonia_aug.pkl','wb') as f:
#     pickle.dump(stack,f)

In [0]:
f1 = h5py.File(path +'added_images_1.h5', 'r')
X1 = f1['dataset_1']
added_images1 = np.array(X1.value)

f1 = h5py.File(path +'added_images_2.h5', 'r')
X1 = f1['dataset_1']
added_images2 = np.array(X1.value)

In [0]:
added_images1 = np.concatenate((added_imas))